<a href="https://colab.research.google.com/github/prodonn/public/blob/main/Bert_based_uncased_on_test_contradictory_mister_watson_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contradictory Mister Watson

This script is the dreamed opportunity to create a translator, then a contradictory classifier with huggingface

## Load the data

In [ ]:
import numpy as np

import tensorflow as tf

import os
import glob
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


Import `matplotlib` and create a helper function to plot graphs:

## What files do I need?

You'll need train.csv, test.csv and sample_submission.csv.

What am I predicting?

You are predicting whether a given tweet is about a real 

    train.csv - the training set
    test.csv - the test set
    sample_submission.csv - a sample submission file in the correct format

Columns

    id - a unique identifier for each premise
    premise - the text of the premise
    hypothesis - the text of the hypothesis
    lang_abv - the abbreviated lang word
    language - The name of the langage
    label - this indicates the result



In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
drive_dir = '/drive/MyDrive'
data_dir = drive_dir + '/nlp_contradictory_watson'
glob.glob(data_dir + '/*.csv')
translated_train_file = data_dir+"/train_translate_df_0_20082021.csv"
train_file = data_dir + "/train.csv"
translated_test_file = data_dir+"/test_translate_df_0_20082021.csv"
test_file = data_dir + "/test.csv"
submission_file =  data_dir + "/sample_submission.csv"

In [ ]:
submission = pd.read_csv(submission_file)

In [ ]:
translated_test_df = pd.read_csv(translated_test_file)
test_df = pd.read_csv(test_file)

In [ ]:
translated_train_df = pd.read_csv(translated_train_file)
train_df = pd.read_csv(train_file)

In [ ]:
test_df.columns

Index(['id', 'premise', 'hypothesis', 'lang_abv', 'language'], dtype='object')

In [ ]:
def extract_dico_value(l_dico) :
  #print('l_dico : ',l_dico)
  dico = l_dico.strip('}][{')
  #print('dico : ', dico)
  dico_1=dico.split('\': ')
  #print(dico_1[1].strip('\'\"'))
  value = dico_1[1].strip('\'\"')
  return value

In [ ]:
translated_test_df['en_premise']=translated_test_df['en_premise'].apply(lambda x: extract_dico_value(x))
translated_test_df['en_hypothesis']=translated_test_df['en_hypothesis'].apply(lambda x: extract_dico_value(x))

In [ ]:
translated_train_df['en_premise']=translated_train_df['en_premise'].apply(lambda x: extract_dico_value(x))
translated_train_df['en_hypothesis']=translated_train_df['en_hypothesis'].apply(lambda x: extract_dico_value(x))

## Lets classify

We 'll use a zero shot classifier from huggingface 

In [ ]:
translated_test_df.head(5)

,id,premise,hypothesis,lang_abv,language,en_premise,en_hypothesis
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...",ur,Urdu,"The box, on the other side, will be hit by box...","What would not have happened to be a memorial,..."
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...,ar,Arabic,That's what we've been advised to do.,"When told what to do, management failed to let..."
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.,fr,French,and this is largely due to the fact that mothe...,Mothers do drugs.
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。,zh,Chinese,Dialogue with representatives of cities and ot...,IMA cooperates with other organizations as the...
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась.",ru,Russian,She was still there.,"We thought she left, but she stayed."


In [ ]:
test_df = pd.merge(test_df,translated_test_df[['id', 'en_premise', 'en_hypothesis']], on='id', how='left')

In [ ]:
test_df['en_premise']=test_df['en_premise'].fillna(test_df['premise'])
test_df['en_hypothesis']=test_df['en_hypothesis'].fillna(test_df['hypothesis'])
test_df.head(10)

,id,premise,hypothesis,lang_abv,language,en_premise,en_hypothesis
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...",ur,Urdu,"The box, on the other side, will be hit by box...","What would not have happened to be a memorial,..."
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...,ar,Arabic,That's what we've been advised to do.,"When told what to do, management failed to let..."
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.,fr,French,and this is largely due to the fact that mothe...,Mothers do drugs.
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。,zh,Chinese,Dialogue with representatives of cities and ot...,IMA cooperates with other organizations as the...
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась.",ru,Russian,She was still there.,"We thought she left, but she stayed."
5,aa2510d454,His family had lost a son and a daughter now.,The son and daughter had lost their father.,en,English,His family had lost a son and a daughter now.,The son and daughter had lost their father.
6,865d1c7b16,Steps are initiated to allow program board mem...,There's enough room for 35-40 positions on the...,en,English,Steps are initiated to allow program board mem...,There's enough room for 35-40 positions on the...
7,a16f7ed56b,C'était probablement la première chose dont je...,C'était l'un de mes premiers souvenirs.,fr,French,It was probably the first thing I remembered a...,It was one of my first memories.
8,6d9fa191e6,"agencies' operating trust, enterprise and inte...",Agencies in financial trouble are usually audi...,en,English,"agencies' operating trust, enterprise and inte...",Agencies in financial trouble are usually audi...
9,c156e8fed5,Hakuna aliyejua walipokwenda.,Mafiko yao ilikuwa ni siri,sw,Swahili,Hakuna aliyejua walipokwenda.,Mafiko yao ilikuwa ni siri


In [ ]:
train_df = pd.merge(train_df,translated_train_df[['id', 'en_premise', 'en_hypothesis']], on='id', how='left')

In [ ]:
train_df['en_premise']=train_df['en_premise'].fillna(train_df['premise'])
train_df['en_hypothesis']=train_df['en_hypothesis'].fillna(train_df['hypothesis'])
train_df.head(10)

,id,premise,hypothesis,lang_abv,language,label,en_premise,en_hypothesis
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0,and these comments were considered in formulat...,The rules developed in the interim were put to...
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0,Little things like this make a huge difference...,I was trying to accomplish something.
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0,you know they can't really defend themselves l...,They can't defend themselves because of their ...
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,"In the same way, there are opportunities to ex...",Children can see how different ethnic groups are.
5,ed7d6a1e62,"Bir çiftlikte birisinin, ağıla kapatılmış bu ö...",Çiftlikte insanlar farklı terimler kullanırlar.,tr,Turkish,0,You can hear someone on a farm saying we have ...,"On the farm, people use different terms."
6,5a0f4908a0,ریاست ہائے متحدہ امریکہ واپس آنے پر، ہج ایف بی...,ہیگ کی تفتیش ایف بی آئی اہلکاروں کی طرف سے کی...,ur,Urdu,0,"On the return of the United States, this visit...",Fever’s investigation was examined by F.
7,fdcd1bd867,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.,en,English,2,From Cockpit Country to St. Ann's Bay,From St. Ann's Bay to Cockpit Country.
8,7cfb3d272c,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...,en,English,1,"Look, it's your skin, but you're going to be i...",The boss will fire you if he sees you slacking...
9,8c10229663,Через каждые сто градусов пятна краски меняют ...,Краска изменяется в соответствии с цветом.,ru,Russian,0,Every 100 degrees of paint stains change color...,Paint changes according to color.


In [ ]:
train_dataset_df = train_df[['id','en_premise','en_hypothesis','label']].copy()

In [ ]:
! pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 264 kB 12.8 MB/s 
     |████████████████████████████████| 2.8 MB 46.4 MB/s 
     |████████████████████████████████| 119 kB 48.4 MB/s 
     |████████████████████████████████| 50 kB 7.3 MB/s 
     |████████████████████████████████| 243 kB 51.6 MB/s 
     |████████████████████████████████| 895 kB 34.3 MB/s 
     |████████████████████████████████| 636 kB 51.0 MB/s 
     |████████████████████████████████| 3.3 MB 45.8 MB/s 
     |████████████████████████████████| 1.1 MB 30.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

from transformers import AutoTokenizer


from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf


import numpy as np

checkpoint_0 = "bert-base-uncased"
checkpoint_1 = 'bert-base-multilingual-cased'
checkpoint=checkpoint_0
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_dataset(dataset):
    encoded = tokenizer(
        dataset[["en_premise", "en_hypothesis"]].values.tolist(),
        padding=True,
        max_length=0,
        return_tensors='tf'
    )
    return encoded.data

#tokenized_test = tokenize_dataset(test_df)
#tokenized_train = tokenize_dataset(train_dataset_df)


In [ ]:
def encode_dataset(dataset):
    encoded = tokenizer(
        dataset[["en_premise", "en_hypothesis"]].values.tolist(),
        padding=True,
        max_length=100,
        truncation=True,
        return_tensors='np'
    )
    return encoded.data

tokenized_test = encode_dataset(test_df)
tokenized_train = encode_dataset(train_dataset_df)

In [ ]:
test_df.en_premise.values[0]

'The box, on the other side, will be hit by box in the name of Isaiah, Isaiah, Oila, and the other high school demand, who lost his life before that year.'

In [ ]:
test_df.en_hypothesis.values[0]

'What would not have happened to be a memorial, one of the high school students died.'

In [ ]:
tokenized_test

{'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 'input_ids': array([[ 101, 1996, 3482, ...,    0,    0,    0],
        [ 101, 2008, 1005, ...,    0,    0,    0],
        [ 101, 1998, 2023, ...,    0,    0,    0],
        ...,
        [ 101, 2014, 2556, ...,    0,    0,    0],
        [ 101, 3475, 1005, ...,    0,    0,    0],
        [ 101, 1999, 4291, ...,    0,    0,    0]]),
 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])}

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(
    tokenized_train['input_ids'], train_df.label.values, 
    test_size=0.2, random_state=2020
)

x_test = tokenized_test['input_ids']

In [ ]:
auto = tf.data.experimental.AUTOTUNE
batch_size=32
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(auto)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(batch_size)
    .cache()
    .prefetch(auto)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(batch_size)
)

In [ ]:
train_dataset

<PrefetchDataset shapes: ((None, 100), (None,)), types: (tf.int64, tf.int64)>

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from transformers import TFAutoModel
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

# First load the transformer layer
transformer_encoder = TFAutoModel.from_pretrained(checkpoint)

# This will be the input tokens 
input_ids = Input(shape=(100,), dtype=tf.int32, name="input_ids")

# Now, we encode the text using the transformers we just loaded
sequence_output = transformer_encoder(input_ids)[0]

# Only extract the token used for classification, which is <s>
cls_token = sequence_output[:, 0, :]

# Finally, pass it through a 3-way softmax, since there's 3 possible laels
out = Dense(3, activation='softmax')(cls_token)

# It's time to build and compile the model
model = Model(inputs=input_ids, outputs=out)
model.compile(
    Adam(lr=1e-5), 
    loss=SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

model.summary()

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
n_steps = len(x_train) // batch_size
n_epochs = 10

train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=n_epochs
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


303/303 [==============================] - 398s 1s/step - loss: 1.0556 - accuracy: 0.4256 - val_loss: 0.9107 - val_accuracy: 0.5866
Epoch 2/10
303/303 [==============================] - 376s 1s/step - loss: 0.8208 - accuracy: 0.6247 - val_loss: 0.8200 - val_accuracy: 0.6498
Epoch 3/10
303/303 [==============================] - 377s 1s/step - loss: 0.6560 - accuracy: 0.7210 - val_loss: 0.8127 - val_accuracy: 0.6667
Epoch 4/10
303/303 [==============================] - 376s 1s/step - loss: 0.4861 - accuracy: 0.7986 - val_loss: 0.9068 - val_accuracy: 0.6658
Epoch 5/10
303/303 [==============================] - 377s 1s/step - loss: 0.3305 - accuracy: 0.8696 - val_loss: 1.0562 - val_accuracy: 0.6720
Epoch 6/10
303/303 [==============================] - 377s 1s/step - loss: 0.2283 - accuracy: 0.9119 - val_loss: 1.2577 - val_accuracy: 0.6708
Epoch 7/10
303/303 [==============================] - 377s 1s/step - loss: 0.1479 - accuracy: 0.9465 - val_loss: 1.4955 - val_accuracy: 0.6716
Epoch 8/10

In [ ]:
hist = train_history.history

In [ ]:
hist

{'accuracy': [0.42563945055007935,
  0.624690592288971,
  0.7210189700126648,
  0.7985767126083374,
  0.8696369528770447,
  0.9119224548339844,
  0.9464727640151978,
  0.9589521288871765,
  0.9692656993865967,
  0.9755569100379944],
 'loss': [1.0556102991104126,
  0.8207581043243408,
  0.6559505462646484,
  0.4861202538013458,
  0.33051738142967224,
  0.22834846377372742,
  0.14794541895389557,
  0.11056186258792877,
  0.08467398583889008,
  0.06940233707427979],
 'val_accuracy': [0.5866336822509766,
  0.6497524976730347,
  0.6666666865348816,
  0.6658415794372559,
  0.6720296740531921,
  0.6707921028137207,
  0.6716171503067017,
  0.6765676736831665,
  0.6679043173789978,
  0.6608911156654358],
 'val_loss': [0.9107102751731873,
  0.8199771642684937,
  0.8127491474151611,
  0.9067670702934265,
  1.0561585426330566,
  1.2577444314956665,
  1.4954853057861328,
  1.486546277999878,
  1.6392052173614502,
  1.700758934020996]}

In [ ]:
import plotly.express as px
px.line(
    hist,  y=['accuracy', 'val_accuracy'], 
    title='Model Accuracy', labels={'x': 'Epoch', 'value': 'Accuracy'}
)

In [ ]:
px.line(
    hist, x=range(1, len(hist['loss'])+1), y=['loss', 'val_loss'], 
    title='Model Loss', labels={'x': 'Epoch', 'value': 'Loss'}
)

ValueError: ignored

In [ ]:
test_preds = model.predict(test_dataset, verbose=1)
submission['prediction'] = test_preds.argmax(axis=1)


163/163 [==============================] - 61s 376ms/step


In [ ]:
submission.head()

,id,prediction
0,c6d58c3f69,1
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,2


In [ ]:
submission.to_csv(data_dir + "/submission_0_10092021.csv", index = False)